In [3]:
from sklearn.linear_model import LogisticRegression
from pathlib import Path
import pandas as pd


In [33]:
df = pd.read_csv(Path('runner_result_runner_result.csv'))
df = df.drop(columns="Unnamed: 0")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13188 entries, 0 to 13187
Data columns (total 89 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   meetingName                      13188 non-null  object 
 1   meetingDate                      13188 non-null  object 
 2   raceNumber                       13188 non-null  int64  
 3   runnerNumber                     13188 non-null  int64  
 4   location                         0 non-null      float64
 5   weatherCondition                 13022 non-null  object 
 6   trackCondition                   13022 non-null  object 
 7   raceName                         13188 non-null  object 
 8   raceStartTime                    13188 non-null  object 
 9   raceDistance                     13188 non-null  int64  
 10  trackDirection                   0 non-null      float64
 11  raceClassConditions              13188 non-null  object 
 12  runnerName        

In [34]:
runner_starts_df = df.iloc[:, 17:31]
runner_wins_df = df.iloc[:,31:45]
runner_placings_df = df.iloc[:,45:59]
trainer_starts_df = df.iloc[:,59:64]
trainer_wins_df = df.iloc[:,64:69]
trainer_placings_df = df.iloc[:,69:74]
rider_starts_df = df.iloc[:,74:79]
rider_wins_df = df.iloc[:,79:84]
rider_placings_df = df.iloc[:,84:]
fixed_wins_df = df.iloc[:,15:17] 

runners_df = pd.concat([runner_starts_df,runner_wins_df,runner_placings_df],axis=1,join='inner')
trainer_df = pd.concat([trainer_starts_df,trainer_wins_df,trainer_placings_df],axis=1,join='inner')
rider_df = pd.concat([rider_starts_df,rider_wins_df,rider_placings_df],axis=1,join='inner') 

df = df.iloc[:,0:15]

In [35]:
df['finishingPosition'] = df['finishingPosition'].apply( lambda x: 1 if x>=1 and x<=3 else 0 )
df.finishingPosition.value_counts()

0    10435
1     2753
Name: finishingPosition, dtype: int64

In [37]:
y = df.finishingPosition

In [38]:
# Convert categorical features to numerical
weatherCondition_df = pd.get_dummies(df.weatherCondition)
trackCondition_df = pd.get_dummies(df.trackCondition)
raceClassConditions_df = pd.get_dummies(df.raceClassConditions)

In [39]:
df = df.drop(columns=['weatherCondition','trackCondition','location','trackDirection','runnerName','riderName'])
df = df.drop(columns=['meetingDate','raceStartTime','raceClassConditions','raceName','meetingName'])
# X = df.drop(columns='finishingPosition')
df = df.drop(columns='finishingPosition')
X = df.drop(columns=['raceNumber','runnerNumber'])

X.value_counts()

raceDistance
1200            2097
1400            1809
1100            1603
1000            1345
1600            1140
                ... 
870                8
3600               8
3800               8
1560               7
2218               6
Length: 109, dtype: int64

In [76]:
def logisticRegression(X,y,include_dfs):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import confusion_matrix
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    
    
    if len(include) > 0:
        for data_frame in include_dfs:
            X = pd.concat([X, data_frame],axis=1,join='inner')

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    X_scaler = StandardScaler()
    X_scaler.fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    # classifier = LogisticRegression(solver='lbfgs', random_state=1)
    classifier = LogisticRegression(solver='newton-cholesky', random_state=1)
    # classifier.fit(X_train, y_train)
    classifier.fit(X_train_scaled, y_train)
    
    # print(f"Training Data Score: {classifier.score(X_train, y_train)}")
    # print(f"Testing Data Score: {classifier.score(X_test, y_test)}")
    # print("------------------")
    
    predictions = classifier.predict(X_test)
    predictions_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test})#.tail(10)

#     # Confusion Matrix
#     confusion_matrix(y_test, predictions)
    
    from sklearn.metrics import accuracy_score
    # Display the accuracy score for the test dataset.
    accuracy_score(y_test, predictions)

    return predictions_df
    

In [78]:
# runners_df = pd.concat([runner_starts_df,runner_wins_df,runner_placings_df],axis=1,join='inner')
# runner_starts_df
# runner_wins_df
# runner_placings_df

# trainer_df = pd.concat([trainer_starts_df,trainer_wins_df,trainer_placings_df],axis=1,join='inner')
# trainer_starts_df
# trainer_wins_df
# trainer_placings_df

# rider_df = pd.concat([rider_starts_df,rider_wins_df,rider_placings_df],axis=1,join='inner') 
# rider_starts_df
# rider_wins_df
# rider_placings_df

# fixed_wins_df
# df = df.iloc[:,0:15]

# weatherCondition_df
# trackCondition_df
# raceClassConditions_df

include_dfs = [runners_df]#,trainer_df,rider_df,weatherCondition_df,trackCondition_df,raceClassConditions_df]
predictions_df = logisticRegression(X,y,include_dfs)
predictions_df.value_counts()

C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Prediction  Actual
1           0         2619
            1          678
dtype: int64